In [1]:
import FinanceDataReader as fdr
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.3f}'.format

### 가설 - 가격 변동성이 크고 거래량이 몰린 종목이 주가가 상승한다.   
"가격 변동성이 크고 거래량이 몰린 종목이 주가가 상승한다" 라는 가설을 증명하기 위해서는 "가격 변동성이 크다", "거래량이 몰린다" 등을 표현하는 변수가 필요합니다. 먼저 일봉데이터를 불러옵니다.

In [5]:
mdl_data = pd.read_pickle('mdl_data.pkl')
mdl_data.head()

,open,high,low,close,volume,change,code,name,kosdaq_return,return,win_market,close_r1,close_r2,close_r3,close_r4,close_r5,max_close
date,,,,,,,,,,,,,,,,,
2021-01-04,2185,2320,2135,2260,588133,0.044,060310,3S,NaN,NaN,0,0.996,1.013,1.013,0.993,0.962,1.013
2021-01-05,2270,2285,2200,2250,410263,-0.004,060310,3S,1.008,0.996,0,1.018,1.018,0.998,0.967,0.971,1.018
2021-01-06,2225,2310,2215,2290,570349,0.018,060310,3S,0.996,1.018,1,1.000,0.980,0.950,0.954,0.950,1.000
2021-01-07,2290,2340,2240,2290,519777,0.000,060310,3S,1.008,1.000,0,0.980,0.950,0.954,0.950,0.959,0.980
2021-01-08,2300,2315,2225,2245,462568,-0.020,060310,3S,0.999,0.980,0,0.969,0.973,0.969,0.978,0.973,0.978


In [104]:
mdl_data['code'].sample(1)[0]

'100700'

첫 번째 종목 060310 에 대하여 가격 변동성 변수를 만들어 보겠습니다. 

In [322]:
kosdaq_list = pd.read_pickle('kosdaq_list.pkl')

data_all = pd.DataFrame()

for code in kosdaq_list['code']:

    data = mdl_data[mdl_data['code']==code].sort_index().copy()
    data['price_mean'] = data['close'].rolling(20).mean()
    data['price_std'] = data['close'].rolling(20).std(ddof=0)
    data['price_z'] = (data['close'] - data['price_mean'])/data['price_std']    
    
    data['volume_mean'] = data['volume'].rolling(20).mean()
    data['volume_std'] = data['volume'].rolling(20).std(ddof=0)
    data['volume_z'] = (data['close'] - data['volume_mean'])/data['volume_std']

       
    data['max_close']  = data[['close_r1','close_r2','close_r3','close_r4','close_r5']].max(axis=1) # 5 영업일 종가 수익율 중 최고 값
    data.dropna(subset=['price_z','volume_z','close_r1','close_r2','close_r3','close_r4','close_r5'], inplace=True) # missing 이 있는 행은 제거  
    
    data = data[(data['price_std']>1) & (data['volume_std']>1)]
    
    data_all = pd.concat([data, data_all], axis=0)



In [323]:
rank1  = pd.qcut(data_all['price_z'], q=5, labels=range(5))
rank2  = pd.qcut(data_all['volume_z'], q=5, labels=range(5))

data_all.groupby([rank1, rank2])['max_close'].mean().unstack()

volume_z,0,1,2,3,4
price_z,,,,,
0,1.037,1.036,1.036,1.036,1.033
1,1.030,1.030,1.030,1.031,1.028
2,1.028,1.028,1.033,1.032,1.028
3,1.029,1.031,1.038,1.039,1.032
4,1.032,1.034,1.042,1.049,1.040


In [320]:
rank1  = pd.qcut(data_all['price_z'], q=5, labels=range(5))
rank2  = pd.qcut(data_all['volume_z'], q=5, labels=range(5))

data_all.groupby([rank1, rank2])['max_close'].mean().unstack()

volume_z,0,1,2,3,4
price_z,,,,,
0,1.035,1.037,1.037,1.036,1.031
1,1.031,1.033,1.032,1.032,1.030
2,1.031,1.032,1.034,1.036,1.031
3,1.030,1.031,1.033,1.039,1.033
4,1.030,1.031,1.037,1.044,1.040
